<h3> Loading the libraries <h3>

In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import json


<h3> Read the data <h3>

In [5]:
data =  pd.read_csv('wtb-internship-test.csv')
print(data.head())

                  Domain name            Company name Twitter  \
0         chateau-giscours.fr        Château Giscours     NaN   
1                    raimo.fr           Raimo Glacier     NaN   
2  champagne-voirin-jumel.com  Champagne Voirin Jumel     NaN   
3        chateaudepoisses.com      Château d'Époisses     NaN   
4              parenchere.com   Château de Parenchère     NaN   

              Facebook  
0                  NaN  
1         raimoglacier  
2                  NaN  
3     chateaudepoisses  
4  chateaudeparenchere  


<h3> Inspecting the data <h3>

In [6]:
data.info()
data.describe()
data.isnull().sum()
data.duplicated().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2022 entries, 0 to 2021
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Domain name   2022 non-null   object
 1   Company name  1940 non-null   object
 2   Twitter       446 non-null    object
 3   Facebook      1288 non-null   object
dtypes: object(4)
memory usage: 63.3+ KB


0

<h3> Fill missing values <h3>

In [7]:

data['Twitter'].fillna('unknown', inplace=True)
data['Facebook'].fillna('unknown', inplace=True)


/var/folders/h4/kdky5skx0jggltns31j4zj7r0000gn/T/ipykernel_72171/1236659601.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Twitter'].fillna('unknown', inplace=True)
/var/folders/h4/kdky5skx0jggltns31j4zj7r0000gn/T/ipykernel_72171/1236659601.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

<h3> Normalize the text <h3>

In [8]:
data['Company name'] = data['Company name'].str.lower().str.strip()
data['Twitter'] = data['Twitter'].str.lower().str.strip()
data['Facebook'] = data['Facebook'].str.lower().str.strip()



<h3> Remove duplicates <h3>

In [9]:
data.drop_duplicates(inplace=True)

<h3> Validate domain names <h3>

In [10]:
import re
def validate_domain(domain):
    pattern = re.compile(r'^(?:[a-z0-9](?:[a-z0-9-]{0,61}[a-z0-9])?\.)+[a-z]{2,6}$')
    return bool(pattern.match(domain))
data['Domain_valid'] = data['Domain name'].apply(validate_domain)


<h3> Validate social media  names <h3>

In [11]:
def validate_twitter(handle):
    pattern = re.compile(r'^@?(\w){1,15}$')
    return bool(pattern.match(handle))

def validate_facebook(handle):
    pattern = re.compile(r'^[a-z\d.]{5,}$')
    return bool(pattern.match(handle))

data['Twitter_valid'] = data['Twitter'].apply(validate_twitter)
data['Facebook_valid'] = data['Facebook'].apply(validate_facebook)





In [17]:
def get_security_data (url):
 audit_data=[]
 row = data.loc[data['Domain name'] == url]
 lighthouse_report=row.iloc[0]['lighthouse_report']
 security_categories = [
        "is-on-https",
        "redirects-http",
        "no-vulnerable-libraries",
        "csp-xss",
        "password-inputs-can-be-pasted-into",
        "content-width",
    ]  # Add more as needed

 audit_data.append( lighthouse_report["audits"]['legacy-javascript'])
 audit_data.append( lighthouse_report["audits"]['redirects-http'])
 audit_data.append( lighthouse_report["audits"]["password-inputs-can-be-pasted-into"])
 

       
 return audit_data         

    

In [ ]:
import requests
from bs4 import BeautifulSoup
import response
def extract_contact_info(url):
    # Method 1: Parsing HTML
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    contact_info = set()  # Using a set to avoid duplicates

    # Extract contact info from HTML elements
    for element in soup.find_all(['p', 'div', 'span']):
        text = element.get_text().strip() 
        # Example pattern for email address extraction
        email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
        emails = re.findall(email_pattern, text)
        for email in emails:
            if ' ' not in email:  # Check if email contains no space
                contact_info.add(email)
        
        # Regular expression pattern for French phone numbers
        phone_pattern = r'(?:\+|00)33\s*\(?(?:0|\(0\))\)?\s*[1-9](?:[\s.-]*\d{2}){4}'
        french_phone_numbers = re.findall(phone_pattern, text)
        contact_info.update(french_phone_numbers)

    return contact_info



url= 'https://beauxvillages.be'
contact_info = extract_contact_info(url)

   
